In [10]:
#good
import sys
import os
import math
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from einops import rearrange, repeat, einsum

import numpy as np
import pandas as pd
import os
#import torchcd 
from torch import nn
from src.learner import Learner
from src.callback.core import *
from src.callback.tracking import *
from src.callback.scheduler import *
from src.callback.patch_mask import *
from src.callback.transforms import *
from src.metrics import *
from datautils import get_dls

from keras.models import Sequential
from keras.layers import LSTM, Dense

import time
import random
import argparse
import datetime
from functools import partial
from sklearn.metrics import mean_squared_error, mean_absolute_error

import torch.backends.cudnn as cudnn
import torch.distributed as dist

from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
from timm.utils import accuracy, AverageMeter

from sklearn.model_selection import train_test_split
from torch.cuda.amp import GradScaler, autocast

from xlstm1.xlstm_block_stack import xLSTMBlockStack, xLSTMBlockStackConfig

from xlstm1.blocks.mlstm.block import mLSTMBlockConfig
from xlstm1.blocks.slstm.block import sLSTMBlockConfig

from myxlstm import xlstm

from typing import List

import numpy as np
import pandas as pd
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset

In [11]:
# Redefine the time feature classes and functions

class TimeFeature:
    def __init__(self):
        pass

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        pass

    def __repr__(self):
        return self.__class__.__name__ + "()"


class SecondOfMinute(TimeFeature):
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.second / 59.0 - 0.5


class MinuteOfHour(TimeFeature):
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.minute / 59.0 - 0.5


class HourOfDay(TimeFeature):
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.hour / 23.0 - 0.5


class DayOfWeek(TimeFeature):
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.dayofweek / 6.0 - 0.5


class DayOfMonth(TimeFeature):
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.day - 1) / 30.0 - 0.5


class DayOfYear(TimeFeature):
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.dayofyear - 1) / 365.0 - 0.5


class MonthOfYear(TimeFeature):
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.month - 1) / 11.0 - 0.5


class WeekOfYear(TimeFeature):
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.isocalendar().week - 1) / 52.0 - 0.5


def time_features_from_frequency_str(freq_str: str) -> List[TimeFeature]:
    features_by_offsets = {
        offsets.YearEnd: [],
        offsets.QuarterEnd: [MonthOfYear],
        offsets.MonthEnd: [MonthOfYear],
        offsets.Week: [DayOfMonth, WeekOfYear],
        offsets.Day: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.BusinessDay: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Hour: [HourOfDay, DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Minute: [
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
        offsets.Second: [
            SecondOfMinute,
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
    }

    offset = to_offset(freq_str)

    for offset_type, feature_classes in features_by_offsets.items():
        if isinstance(offset, offset_type):
            return [cls() for cls in feature_classes]

    supported_freq_msg = f"""
    Unsupported frequency {freq_str}
    The following frequencies are supported:
        Y   - yearly
            alias: A
        M   - monthly
        W   - weekly
        D   - daily
        B   - business days
        H   - hourly
        T   - minutely
            alias: min
        S   - secondly
    """
    raise RuntimeError(supported_freq_msg)
def time_features(dates, freq='D'):
    return np.vstack([feat(dates) for feat in time_features_from_frequency_str(freq)])

In [12]:
data = pd.read_csv('D:/ĐACN/xLSTMTime-main/Datasets/forex/EURUSD_03_23.csv',index_col='date', parse_dates=True)

In [13]:
from sklearn.preprocessing import StandardScaler
time_feats = time_features(data.index, 'D')  # Generate time features assuming daily frequency
time_features_names = [ 'day_of_week', 'day_of_month', 'day_of_year']
time_features_df = pd.DataFrame(time_feats.T, columns=time_features_names, index=data.index)
data_with_time_features = pd.concat([data, time_features_df], axis=1)

# Normalize the features
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data_with_time_features)

# Display the first few rows of the new dataframe with time features added
data_with_time_features.head()

,open,high,low,close,volume,MA_10,MACD,MACD_signal,ROC_2,Momentum_4,RSI_10,BB_high,BB_low,BB_middle,CCI_20,day_of_week,day_of_month,day_of_year
date,,,,,,,,,,,,,,,,,,
2003-01-01,1.0492,1.0503,1.0467,1.0480,601,1.03691,0.011034,0.008843,0.038183,0.0112,84.505589,1.054167,0.998433,1.026300,141.214955,-0.166667,-0.500000,-0.500000
2003-01-02,1.0479,1.0500,1.0335,1.0366,8880,1.03786,0.010255,0.009125,-1.172657,-0.0077,59.073128,1.053598,1.002632,1.028115,83.305890,0.000000,-0.466667,-0.497260
2003-01-03,1.0362,1.0440,1.0340,1.0428,9109,1.03940,0.010023,0.009305,-0.496183,-0.0048,65.370919,1.054475,1.005125,1.029800,75.442338,0.166667,-0.433333,-0.494521
2003-01-06,1.0420,1.0497,1.0416,1.0447,9405,1.04121,0.009878,0.009420,0.781401,-0.0042,67.095012,1.055373,1.007607,1.031490,97.651698,-0.500000,-0.333333,-0.486301
2003-01-07,1.0444,1.0457,1.0395,1.0405,9297,1.04223,0.009317,0.009399,-0.220560,-0.0075,59.784293,1.054621,1.011649,1.033135,66.162912,-0.333333,-0.300000,-0.483562


In [14]:
from sklearn.model_selection import train_test_split

# Define the split sizes
train_size = 0.80
validation_size = 0.15
test_size = 0.05

# Calculate the actual split sizes for validation and test relative to remaining data after train split
validation_size_adjusted = validation_size / (1 - train_size)

# Split the data into train and temporary set (validation + test)
data_train, data_temp = train_test_split(data_with_time_features, train_size=train_size, shuffle=False)

# Split the temporary set into validation and test
data_validation, data_test = train_test_split(data_temp, train_size=validation_size_adjusted, shuffle=False)

# Check the sizes of each dataset
data_train.shape, data_validation.shape, data_test.shape


((4432, 18), (831, 18), (278, 18))

In [15]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.functional import mse_loss, l1_loss

# Function to create Tensors from numpy arrays for PyTorch
def create_tensors(data):
    return torch.tensor(data, dtype=torch.float32)

# Reshape data and convert to PyTorch tensors
X_train = create_tensors(data_train.drop('close', axis=1).values).reshape(-1, 1, data_train.shape[1] - 1)
y_train = create_tensors(data_train['close'].values).reshape(-1, 1)
X_validation = create_tensors(data_validation.drop('close', axis=1).values).reshape(-1, 1, data_validation.shape[1] - 1)
y_validation = create_tensors(data_validation['close'].values).reshape(-1, 1)
X_test = create_tensors(data_test.drop('close', axis=1).values).reshape(-1, 1, data_test.shape[1] - 1)
y_test = create_tensors(data_test['close'].values).reshape(-1, 1)

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Model instantiation
model = LSTMModel(input_dim=X_train.shape[2], hidden_dim=50)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.01)

# Define batch size
batch_size = 32  # You can adjust this value as needed

# Prepare data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = TensorDataset(X_validation, y_validation)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# Training the model
def train_model(num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluate the model
def evaluate_model():
    model.eval()
    with torch.no_grad():
        val_losses = []
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_losses.append(loss.item())
        avg_loss = sum(val_losses) / len(val_losses)
    print(f'Validation Loss: {avg_loss}')

def evaluate_model_on_test():
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        test_losses_mse = []
        test_losses_mae = []
        
        # Prepare the test loader
        test_dataset = TensorDataset(X_test, y_test)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        for inputs, targets in test_loader:
            outputs = model(inputs)
            
            # Calculate MSE and MAE
            loss_mse = mse_loss(outputs, targets)
            loss_mae = l1_loss(outputs, targets)
            
            test_losses_mse.append(loss_mse.item())
            test_losses_mae.append(loss_mae.item())
        
        # Calculate average losses
        avg_mse = sum(test_losses_mse) / len(test_losses_mse)
        avg_mae = sum(test_losses_mae) / len(test_losses_mae)
    
    print(f'Test MSE: {avg_mse}')
    print(f'Test MAE: {avg_mae}')

# Run training and evaluation
train_model(50)
evaluate_model_on_test()


Epoch 1, Loss: 0.014228549785912037
Epoch 2, Loss: 0.01576698198914528
Epoch 3, Loss: 0.013516672886908054
Epoch 4, Loss: 0.010585346259176731
Epoch 5, Loss: 0.012935062870383263
Epoch 6, Loss: 0.011925282888114452
Epoch 7, Loss: 0.008378230966627598
Epoch 8, Loss: 0.019651122391223907
Epoch 9, Loss: 0.011906091123819351
Epoch 10, Loss: 0.018057970330119133
Epoch 11, Loss: 0.00950110238045454
Epoch 12, Loss: 0.008659851737320423
Epoch 13, Loss: 0.009669283404946327
Epoch 14, Loss: 0.00932097528129816
Epoch 15, Loss: 0.01341534499078989
Epoch 16, Loss: 0.013502925634384155
Epoch 17, Loss: 0.012190116569399834
Epoch 18, Loss: 0.02649473026394844
Epoch 19, Loss: 0.020379364490509033
Epoch 20, Loss: 0.017293579876422882
Epoch 21, Loss: 0.018149886280298233
Epoch 22, Loss: 0.013418524526059628
Epoch 23, Loss: 0.01854548044502735
Epoch 24, Loss: 0.02315455675125122
Epoch 25, Loss: 0.027532897889614105
Epoch 26, Loss: 0.010997777804732323
Epoch 27, Loss: 0.014208221808075905
Epoch 28, Loss: 0

In [3]:
import argparse
import importlib

def check_library_version(library_name):
    try:
        lib = importlib.import_module(library_name)
        print(f"The version of {library_name} is {lib.__version__}")
    except ImportError:
        print(f"Library {library_name} is not installed.")
    except AttributeError:
        print(f"Version information is not available for {library_name}.")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Check the version of a Python library.")
    parser.add_argument("library", type=str, help="Name of the library to check")
    
    args = parser.parse_args()
    check_library_version(args.library)


usage: ipykernel_launcher.py [-h] library
ipykernel_launcher.py: error: the following arguments are required: library


SystemExit: 2

c:\Users\Admin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
!pip show tensorflow

Name: tensorflow
Version: 2.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\Admin\anaconda3\Lib\site-packages
Requires: tensorflow-intel
Required-by: 
